In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 1.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.18 requires numpy<2, but you have numpy 2.1.3 which is incompatible.
mediapipe 0.10.18 requires protobuf<5,>=4.25.3, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.


In [6]:
import requests
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Lucknow locations with proper coordinates
LOCATIONS = {
    "Depot": (26.8467, 80.9462),  # Central Lucknow
    "Alambagh": (26.8296, 80.8910),
    "Ashiyana": (26.8636, 80.9901),
    "Hazratganj": (26.8543, 80.9487),
    "Gomti Nagar": (26.8391, 80.9906),
    "Indira Nagar": (26.8753, 81.0111),
    "Munshipulia": (26.8989, 81.0289),
    "Rajajipuram": (26.8021, 80.9015),
    "Faizabad Road": (26.7815, 82.1325)
}

ORDERS = [
    (1, "Alambagh", "Ashiyana"),
    (2, "Hazratganj", "Gomti Nagar"),
    (3, "Alambagh", "Rajajipuram"),
    (4, "Gomti Nagar", "Faizabad Road")
]

def create_data_model():
    """Create problem data with Lucknow parameters"""
    data = {}
    data['locations'] = list(LOCATIONS.values())  # All locations including depot first
    data['pickups_deliveries'] = []
    data['order_ids'] = []
    
    for order_id, pickup_loc, drop_loc in ORDERS:
        data['pickups_deliveries'].append((
            data['locations'].index(LOCATIONS[pickup_loc]),
            data['locations'].index(LOCATIONS[drop_loc])
        ))
        data['order_ids'].append(order_id)
    
    data['num_vehicles'] = 1
    data['depot'] = 0
    data['vehicle_capacity'] = 20  # kg
    return data

def get_osrm_duration_matrix(locations):
    """Get real travel times from OSRM"""
    coords = ';'.join(f"{lon},{lat}" for lat, lon in locations)
    url = f"http://router.project-osrm.org/table/v1/driving/{coords}?annotations=duration"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()['durations']
    except Exception as e:
        # Fallback: generate a dummy time matrix if OSRM fails
        return [[0 for _ in range(len(locations))] for _ in range(len(locations))]

def print_solution(data, manager, routing, solution):
    """Print Lucknow-specific delivery instructions"""
    print("\nOptimal Delivery Route for Lucknow Rider:")
    route_steps = []
    
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        
        if 0 < node < len(data['locations']):
            # Find corresponding order
            order_index = (node - 1) // 2
            if order_index < len(data['order_ids']):
                order_id = data['order_ids'][order_index]
                
                if node % 2 == 1:
                    route_steps.append(f"PICK UP order {order_id}")
                else:
                    route_steps.append(f"DROP order {order_id}")
        
        index = solution.Value(routing.NextVar(index))
    
    for i, step in enumerate(route_steps, 1):
        print(f"{i}. {step}")

def main():
    data = create_data_model()
    
    # Generate duration matrix
    time_matrix = get_osrm_duration_matrix(data['locations'])
    
    # Create routing index manager
    manager = pywrapcp.RoutingIndexManager(
        len(data['locations']), data['num_vehicles'], data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)

    # Time callback with safer indexing
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return time_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Add Time dimension
    routing.AddDimension(
        transit_callback_index,
        3600,  # Allow waiting time (1 hour)
        3600*2,  # Maximum route duration (2 hours)
        False,  # Don't force start cumul to zero
        'Time'
    )
    
    # Add Capacity dimension
    def capacity_callback(from_index):
        """Assign load based on node type"""
        from_node = manager.IndexToNode(from_index)
        return 1 if 0 < from_node < len(data['locations']) and from_node % 2 == 1 else 0

    capacity_callback_index = routing.RegisterUnaryTransitCallback(capacity_callback)
    routing.AddDimensionWithVehicleCapacity(
        capacity_callback_index,
        0,  # Null capacity slack
        [data['vehicle_capacity']],  # Vehicle capacities
        True,  # Start cumul to zero
        'Capacity'
    )
    
    # Add Pickup-Delivery constraints
    for pickup, delivery in data['pickups_deliveries']:
        routing.AddPickupAndDelivery(pickup, delivery)
    
    # Search parameters
    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )
    search_params.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_params.time_limit.seconds = 5

    # Solve the problem
    solution = routing.SolveWithParameters(search_params)
    
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found!")

if __name__ == "__main__":
    main()


Optimal Delivery Route for Lucknow Rider:
1. PICK UP order 2
2. PICK UP order 1
3. DROP order 3
4. DROP order 2
5. DROP order 1
6. PICK UP order 3
7. DROP order 4
8. PICK UP order 4


# ADD WEIGHT

In [8]:
import requests
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import math

# Lucknow locations with proper coordinates
LOCATIONS = {
    "Depot": (26.8467, 80.9462),  # Central Lucknow
    "Alambagh": (26.8296, 80.8910),
    "Ashiyana": (26.8636, 80.9901),
    "Hazratganj": (26.8543, 80.9487),
    "Gomti Nagar": (26.8391, 80.9906),
    "Indira Nagar": (26.8753, 81.0111),
    "Munshipulia": (26.8989, 81.0289),
    "Rajajipuram": (26.8021, 80.9015),
    "Faizabad Road": (26.7815, 82.1325)
}

ORDERS = [
    (1, "Alambagh", "Ashiyana", 5),
    (2, "Hazratganj", "Gomti Nagar", 3),
    (3, "Alambagh", "Rajajipuram", 7),
    (4, "Gomti Nagar", "Faizabad Road", 4)
]

def haversine_distance(loc1, loc2):
    """Calculate distance between two coordinates using Haversine formula"""
    R = 6371  # Earth's radius in kilometers
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    distance = R * c
    
    return int(distance * 1000)  # Convert to meters

def create_data_model(max_vehicle_capacity=20):
    """Create problem data model with flexible capacity"""
    data = {}
    data['locations'] = list(LOCATIONS.values())
    data['pickups_deliveries'] = []
    data['order_ids'] = []
    data['order_weights'] = []
    
    for order_id, pickup_loc, drop_loc, weight in ORDERS:
        data['pickups_deliveries'].append((
            data['locations'].index(LOCATIONS[pickup_loc]),
            data['locations'].index(LOCATIONS[drop_loc])
        ))
        data['order_ids'].append(order_id)
        data['order_weights'].append(weight)
    
    data['num_vehicles'] = 1
    data['depot'] = 0
    data['vehicle_capacity'] = max_vehicle_capacity
    return data

def get_distance_matrix(locations):
    """Generate distance matrix using Haversine distance"""
    matrix = [[0] * len(locations) for _ in range(len(locations))]
    for i, loc1 in enumerate(locations):
        for j, loc2 in enumerate(locations):
            if i != j:
                matrix[i][j] = haversine_distance(loc1, loc2)
    return matrix

def print_solution(data, manager, routing, solution, first_solution=False):
    """Print delivery route solutions"""
    if first_solution:
        print("\nFirst Feasible Solution:")
    else:
        print("\nOptimal Solution:")
    
    route_steps = []
    total_distance = 0
    total_weight = 0
    
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        
        if 0 < node < len(data['locations']):
            order_index = (node - 1) // 2
            if order_index < len(data['order_ids']):
                order_id = data['order_ids'][order_index]
                weight = data['order_weights'][order_index]
                
                if node % 2 == 1:
                    route_steps.append(f"PICK UP order {order_id} (Weight: {weight} kg)")
                    total_weight += weight
                else:
                    route_steps.append(f"DROP order {order_id} (Weight: {weight} kg)")
        
        next_index = solution.Value(routing.NextVar(index))
        if not routing.IsEnd(next_index):
            dist = routing.GetArcCostForVehicle(index, next_index, 0)
            total_distance += dist
        
        index = next_index
    
    for i, step in enumerate(route_steps, 1):
        print(f"{i}. {step}")
    
    print(f"\nTotal Route Distance: {total_distance/1000:.2f} km")
    print(f"Total Payload: {total_weight} kg")

def main():
    data = create_data_model()
    distance_matrix = get_distance_matrix(data['locations'])
    
    # Configurations to try
    capacity_configs = [20, 15, 10, 5]
    
    for max_capacity in capacity_configs:
        print(f"\n--- ROUTE OPTIMIZATION (Max Capacity: {max_capacity} kg) ---")
        
        # Update data model with current capacity
        data = create_data_model(max_capacity)
        
        manager = pywrapcp.RoutingIndexManager(
            len(data['locations']), data['num_vehicles'], data['depot']
        )
        routing = pywrapcp.RoutingModel(manager)

        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return distance_matrix[from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        def capacity_callback(from_index):
            from_node = manager.IndexToNode(from_index)
            if 0 < from_node < len(data['locations']) and from_node % 2 == 1:
                order_index = (from_node - 1) // 2
                return data['order_weights'][order_index]
            return 0

        capacity_callback_index = routing.RegisterUnaryTransitCallback(capacity_callback)
        routing.AddDimensionWithVehicleCapacity(
            capacity_callback_index,
            0,
            [max_capacity],
            True,
            'Capacity'
        )
        
        for pickup, delivery in data['pickups_deliveries']:
            routing.AddPickupAndDelivery(pickup, delivery)
        
        search_params = pywrapcp.DefaultRoutingSearchParameters()
        search_params.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
        )
        search_params.local_search_metaheuristic = (
            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
        )
        search_params.time_limit.seconds = 30

        solution = routing.SolveWithParameters(search_params)
        
        if solution:
            print_solution(data, manager, routing, solution)
        else:
            print(f"No solution found for {max_capacity} kg capacity")

if __name__ == "__main__":
    main()


--- ROUTE OPTIMIZATION (Max Capacity: 20 kg) ---

Optimal Solution:
1. PICK UP order 2 (Weight: 3 kg)
2. PICK UP order 1 (Weight: 5 kg)
3. PICK UP order 4 (Weight: 4 kg)
4. DROP order 2 (Weight: 3 kg)
5. DROP order 4 (Weight: 4 kg)
6. DROP order 3 (Weight: 7 kg)
7. PICK UP order 3 (Weight: 7 kg)
8. DROP order 1 (Weight: 5 kg)

Total Route Distance: 249.60 km
Total Payload: 19 kg

--- ROUTE OPTIMIZATION (Max Capacity: 15 kg) ---
No solution found for 15 kg capacity

--- ROUTE OPTIMIZATION (Max Capacity: 10 kg) ---
No solution found for 10 kg capacity

--- ROUTE OPTIMIZATION (Max Capacity: 5 kg) ---
No solution found for 5 kg capacity


# RL 

In [ ]:

# State Space

# Current rider location
# Pending orders
# Vehicle capacity
# Time of day
# Traffic conditions
# Vehicle battery/fuel status

# Action Space

# Route selection
# Order pickup/drop prioritization
# Dynamic order reassignment
# Capacity management

# Reward Structure

# Positive Rewards:

# Successful timely delivery (+10 points)
# Efficient route completion (+5 points)
# Minimizing idle time (+3 points)


# Negative Rewards:

# Delivery delays (-5 points)
# Capacity violations (-7 points)
# Unnecessary route deviations (-3 points)



# RL Training Approach
# Algorithm: Proximal Policy Optimization (PPO) (STUPID ENOUGH TO TRY THIS)

# Handles continuous and discrete action spaces
# Stable learning across complex routing scenarios
# Efficient policy updates

# Training Phases

# Exploration Phase
# Exploitation Phase
# Adaptive Learning

# Performance Metrics

# Delivery Completion Rate
# Average Delivery Time
# Route Efficiency
# Capacity Utilization
# Customer Satisfaction Index


In [ ]:
# import numpy as np
# import pandas as pd
# import osmnx as ox
# import networkx as nx
# from typing import List, Dict, Tuple
# import random
# from scipy.stats import truncnorm
# import gymnasium as gym
# from stable_baselines3 import PPO
# from stable_baselines3.common.vec_env import DummyVecEnv

# class DeliveryEnvironment(gym.Env):
#     def __init__(self, orders: List[Dict], locations: List[Tuple]):
#         super().__init__()
#         self.orders = orders
#         self.locations = locations
        
#         # Define action and observation spaces
#         self.action_space = gym.spaces.Box(
#             low=-1, high=1, shape=(2,), dtype=np.float32
#         )
#         self.observation_space = gym.spaces.Box(
#             low=-np.inf, high=np.inf, shape=(10,), dtype=np.float32
#         )
        
#         self.reset()
    
#     def reset(self):
#         # Initialize environment state
#         self.current_location = self.locations[0]
#         self.remaining_orders = self.orders.copy()
#         self.completed_orders = []
#         return self._get_observation()
    
#     def step(self, action):
#         # Process action and update state
#         reward = 0
#         done = False
        
#         # Select and process order based on action
#         selected_order = self._select_order(action)
        
#         # Update location, process order
#         if selected_order:
#             reward += self._process_order(selected_order)
        
#         # Check termination conditions
#         if not self.remaining_orders:
#             done = True
        
#         return self._get_observation(), reward, done, {}
    
#     def _get_observation(self):
#         # Generate observation vector
#         return np.array([
#             self.current_location[0],  # Latitude
#             self.current_location[1],  # Longitude
#             len(self.remaining_orders),
#             random.random(),  # Simulated traffic condition
#             random.random(),  # Time of day factor
#         ])
    
#     def _select_order(self, action):
#         # Order selection logic based on RL action
#         return None  # Placeholder
    
#     def _process_order(self, order):
#         # Order processing and reward calculation
#         return 0  # Placeholder reward

# def generate_synthetic_orders(num_orders=50):
#     """Generate realistic synthetic delivery orders"""
#     lucknow_locations = [
#         (26.8467, 80.9462),  # Depot
#         (26.8296, 80.8910),  # Alambagh
#         (26.8636, 80.9901),  # Ashiyana
#         # Add more locations
#     ]
    
#     orders = []
#     for i in range(num_orders):
#         pickup = random.choice(lucknow_locations[1:])
#         delivery = random.choice([loc for loc in lucknow_locations[1:] if loc != pickup])
#         weight = truncnorm.rvs(0, 15, loc=5, scale=3)
        
#         orders.append({
#             'id': i+1,
#             'pickup': pickup,
#             'delivery': delivery,
#             'weight': weight
#         })
    
#     return orders

# def train_rl_model(environment):
#     """Train Reinforcement Learning Model"""
#     model = PPO("MlpPolicy", environment, verbose=1)
#     model.learn(total_timesteps=10000)
#     return model

# def main():
#     # Generate Synthetic Data
#     orders = generate_synthetic_orders()
#     locations = [(26.8467, 80.9462)]  # Add more locations
    
#     # Create RL Environment
#     env = DeliveryEnvironment(orders, locations)
    
#     # Train RL Model
#     trained_model = train_rl_model(DummyVecEnv([lambda: env]))
    
#     # Evaluate and simulate
#     obs = env.reset()
#     for _ in range(100):
#         action, _ = trained_model.predict(obs)
#         obs, reward, done, _ = env.step(action)
#         if done:
#             break

# if __name__ == "__main__":
#     main()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd

# class PerformanceMetricsDashboard:
#     def __init__(self, simulation_data):
#         self.data = simulation_data
    
#     def plot_delivery_times(self):
#         plt.figure(figsize=(10, 6))
#         sns.boxplot(x='order_type', y='delivery_time', data=self.data)
#         plt.title('Delivery Time Distribution')
#         plt.show()
    
#     def calculate_kpis(self):
#         """Calculate Key Performance Indicators"""
#         return {
#             'avg_delivery_time': self.data['delivery_time'].mean(),
#             'on_time_percentage': (self.data['on_time'] == True).mean() * 100,
#             'total_revenue': self.data['order_value'].sum(),
#             'rider_utilization': self.data['rider_active_time'].mean()
#         }
    
#     def generate_rider_performance_report(self):
#         """Generate comprehensive rider performance report"""
#         rider_performance = self.data.groupby('rider_id').agg({
#             'delivery_time': 'mean',
#             'on_time': 'mean',
#             'order_value': 'sum'
#         })
#         return rider_performance

# def main():
#     # Simulated performance data
#     performance_data = pd.DataFrame({
#         'rider_id': [1, 2, 3],
#         'delivery_time': [45, 50, 40],
#         'on_time': [True, False, True],
#         'order_value': [1000, 1200, 950],
#         'rider_active_time': [8, 7.5, 8.2]
#     })
    
#     dashboard = PerformanceMetricsDashboard(performance_data)
#     kpis = dashboard.calculate_kpis()
#     print("Performance KPIs:", kpis)

# if __name__ == "__main__":
#     main()

In [ ]:



# Fallback Mechanisms I try now
# 1. Rule-Based Routing (thinking of simple if else along with distance)

# First we go with simple distance-based algorithms (e.g., Nearest Neighbor)
# we gotta prioritize nearest pending orders (greedy approach)
# also implement basic capacity constraints (e.g., weight limits)

# 2. Stochastic Optimization (MCTS bs)

# Monte Carlo simulations
# Probabilistic order assignment
# Quick decision-making under uncertainty

# 3. Hybrid Approach

# Combine deterministic and probabilistic methods
# Adaptive threshold-based routing
# Graceful degradation of performance



# GOTTA IMPLEMENT SOMETING LIKE THIS ; I GUESS ; LETS SEE